## SQL实战 | nowcoder 91题总结
### 第四部分 SQL:81-91题

　　ID：wlong
　　数据分析爱好者、目前小白一枚
　　邮箱：wlong826@163.com
　　和鲸社区：[【wlong】](https://www.heywhale.com/home/user/profile/5f0a61d72be199002c073935)
　　如果有不完善的地方，欢迎小伙伴评论区留言！
　　最近博客、和鲸每周都会有所更新，欢迎大家关注点赞分享哦！
 
 ----
 
 　　这里呢，建议SQL不熟练的小伙伴先去**nowcoder网**上刷一遍，遇到不会的时候再过来看**答案**，总共**80道题**左右，但nowcoder网上有些题目存在**断层不连续**，序号写到了91题，所以这里主要也是以官网的序号为准。刷题链接见这[【数据库SQL实战】](https://www.nowcoder.com/ta/sql) (★★★)

　牛客网SQL刷题链接：[【数据库SQL实战】](https://www.nowcoder.com/ta/sql) (★★★)

　如果对爬虫感兴趣的小伙伴，看这看这👉：[【零基础入门Python爬虫--豆瓣、58同城、安居客等】](https://www.heywhale.com/mw/project/60be1fa8acdcb3001707b133)

　关于Pandas数据处理的一些小总结，看这看这👉：[【Pandas数据处理120题知识点总结】](https://www.heywhale.com/mw/project/60c0b06919d601001898394c)

　经典数分案例泰坦尼克号的生存分析，看这看这👉：[【经典案例之泰坦尼克号乘客生存情况预测分析 】](https://www.heywhale.com/mw/project/60d18448056f570017c149dc)
 
 　关于Hive SQL的一点小小总结，看这看这👉：[【《Hive 编程指南》的小小总结】](https://www.heywhale.com/mw/project/60f00c93be5f0900176366a6)

　如果小伙伴对于Python实现数据结构感兴趣，可以看这👉：[【线性表】](https://www.heywhale.com/mw/project/60e7b58cbe5f09001761ac87)、[【栈和队列】](https://www.heywhale.com/mw/project/60ebe5de3f2448001705ccc2)

### 81. 牛客的课程订单分析(五)(★★★★★)

　　写出一个sql语句查询在2025-10-15以后，如果有一个用户下单2个以及2个以上状态为购买成功的C++课程或Java课程或Python课程，那么输出这个用户的user_id，以及满足前面条件的第一次购买成功的C++课程或Java课程或Python课程的日期first_buy_date，以及满足前面条件的第二次购买成功的C++课程或Java课程或Python课程的日期second_buy_date，以及购买成功的C++课程或Java课程或Python课程的次数cnt，并且输出结果按照user_id升序排序；

In [ ]:
select
a.user_id
,a.first_buy_date
,b.second_buy_date 
,a.cnt
from (
		select
		user_id
		,min(date) first_buy_date
		,count(*) cnt
		from order_info
		where user_id in ( # 先找出来满足条件的用户id
							select user_id
							from order_info
							where date > '2025-10-15'
							and product_name in ('C++', 'Java', 'Python')
							and status = 'completed'
							group by 1
							having count(*) >= 2
						 )
		and date > '2025-10-15'
		and product_name in ('C++', 'Java', 'Python')
		and status = 'completed'
		group by 1
	 ) a
join (
		select
		user_id
		,c.date second_buy_date 
		from (
				select
				user_id
				,date
				,row_number() over(partition by user_id order by date) t_rank
				from order_info
				where date > '2025-10-15'
				and product_name in ('C++', 'Java', 'Python')
				and status = 'completed'
			 ) c
		where c.t_rank = 2
	 ) b
on a.user_id = b.user_id
order by 1;

### 82. 牛客的课程订单分析(六)

　　写出一个sql语句查询在2025-10-15以后，同一个用户下单2个以及2个以上状态为购买成功的C++课程或Java课程或Python课程的订单id，是否拼团以及客户端名字信息，最后一列如果是非拼团订单，则显示对应客户端名字，如果是拼团订单，则显示NULL，并且按照order_info的id升序排序；

In [2]:
select
o.id
,is_group_buy
,c.name client_name
from order_info o
left join client c
on o.client_id = c.id
where user_id in (
					select user_id
					from order_info
					where date > '2025-10-15'
					and status = 'completed'
					and product_name in ('C++', 'Java', 'Python')
					group by 1
					having count(*) >= 2
				 )
and date > '2025-10-15'
and status = 'completed'
and product_name in ('C++', 'Java', 'Python')
order by 1;

### 83. 牛客的课程订单分析(七)(★★★★)

　　写出一个sql语句查询在2025-10-15以后，同一个用户下单2个以及2个以上状态为购买成功的C++课程或Java课程或Python课程的来源信息，第一列是显示的是客户端名字，如果是拼团订单则显示GroupBuy，第二列显示这个客户端(或者是拼团订单)有多少订单，最后结果按照第一列(source)升序排序；

In [ ]:
select
case when o.is_group_buy = 'Yes' then 'GroupBuy' else c.name end source
,count(*) cnt
from order_info o
left join client c
on o.client_id = c.id
where user_id in (
					select user_id
					from order_info
					where date > '2025-10-15'
					and status = 'completed'
					and product_name in ('C++', 'Java', 'Python')
					group by 1
					having count(*) >= 2
				 )
and date > '2025-10-15'
and status = 'completed'
and product_name in ('C++', 'Java', 'Python')
group by 1
order by 1;

### 84. 实习广场投递简历分析(一)

　　写出SQL语句查询在2025年内投递简历的岗位和数量，并且按数量降序排序；

In [ ]:
select
job
,sum(num) cnt
from resume_info
where year(date) = 2025
group by 1
order by 2 desc;

### 85. 实习广场投递简历分析(二)

　　写出SQL语句查询在2025年内投递简历的每个岗位，每一个月内收到简历的数量，并且按先按月份降序排序，再按简历数目降序排序；

In [ ]:
select
job
,substring(date,1,7) mon
,sum(num) cnt
from resume_info
where year(date) = 2025
group by 1,2
order by 2 desc, 3 desc;

### 86. 实习广场投递简历分析(三)(★★★★★)

　　写出SQL语句查询在2025年投递简历的每个岗位，每一个月内收到简历的数目，和对应的2026年的同一个月同岗位，收到简历的数目，最后的结果先按first_year_mon月份降序，再按job降序排序显示；

In [3]:
# 方法一：
select
first.job
,first.year_mon first_year_mon
,first.cnt first_year_cnt
,second.year_mon second_year_mon
,second.cnt second_year_cnt 
from ( # 2025年
		select
		job
		,month(date) mon
		,substring(date,1,7) year_mon # 或者date_format(date, '%Y-%m')
		,sum(num) cnt
		from resume_info
		where year(date) = 2025 # 或者 date like '2026%'
		group by 1,2
	 ) first
join ( # 2026年
		select
		job
		,month(date) mon
		,substring(date,1,7) year_mon
		,sum(num) cnt
		from resume_info
		where year(date) = 2026 # 或者 date like '2026%'
		group by 1,2
	 ) second
on first.mon = second.mon # 或者 right(first_year_mon,2) = right(second_year_mon,2)
and first.job = second.job
order by 2 desc, 1 desc;

In [ ]:
# 方法二：with as创建临时表
with 
r1 as
(
	select 
	id
	,job
	,date
	,date_format(date, '%Y-%m') as first_year_mon
	,sum(num) as first_year_cnt
	from resume_info
	where date like '2025%'
	group by job, first_year_mon
),
r2 as
(
	select
	id
	,job
	,date
	,date_format(date, '%Y-%m') as second_year_mon
	,sum(num) as second_year_cnt
	from resume_info
	where date like '2026%'
	group by job, second_year_mon
)

select r1.job
,first_year_mon
,first_year_cnt
,second_year_mon
,second_year_cnt
from r1 , r2 
where r1.job = r2.job and date_format(r1.date, '%m') = date_format(r2.date, '%m')
order by first_year_mon desc, r1.job desc;

### 87. 最差是第几名(一)

　　写出一个SQL查询，如果一个学生知道了自己综合成绩以后，最差是排第几名? 结果按照grade升序排序；

In [ ]:
select
grade
,sum(number) over(order by grade) t_rank
from class_grade
order by grade;

### 88. 最差是第几名(二)(★★★★)

　　老师想知道学生们综合成绩的中位数是什么档位，请你写SQL帮忙查询一下，如果只有1个中位数，输出1个，如果有2个中位数，按grade升序输出；

In [ ]:
with r as
(
	select sum(number)
	from class_grade
	group by grade
)

select grade
from (
		select 
		grade
		,case when r%2=1 then r/2+0.5 else r/2 end as start
		,case when r%2=1 then r/2+0.5 else r/2+1 end as end
		from class_grade
	 ) a
join (
		select
		grade
		,sum(number) over(order by grade) t_rank
		from class_grade
		order by grade
	 ) b
on a.grade = b.grade
where a.end <= b.t_rank;

### 89. 获得积分最多的人(一)

　　写一个SQL查找总积分最高的用户的名字，以及他的总积分是多少(此题数据保证积分最高的用户有且只有1个)；

In [ ]:
select
name
,sum(grade_num) sum
from grade_info g
join user u
on g.user_id = u.id
group by 1
order by sum desc
limit 1;

### 90. 获得积分最多的人(二)(★★★★)

　　写一个SQL查找积分增加最高的用户的id(可能有多个)，名字，以及他的总积分是多少，查询结果按照id升序排序；

In [ ]:
select
u.id
,u.name
,sum(grade_num) grade_sum
from grade_info g
join user u
on g.user_id = u.id
where type = 'add'
group by 1,2
having grade_sum >= ALL(
						select sum(grade_num)	
						from grade_info
						where type = 'add'
						group by user_id
					  );

### 91. 获得积分最多的人(三)(★★★★★)

　　写一个SQL查找积分最高的用户的id，名字，以及他的总积分是多少(可能有多个)，查询结果按照id升序排序；

In [ ]:
with table1 as
(
	select
	id
	,name
	,sum(case when type = 'add' then grade_num else -grade_num end) grade_sum
	from grade_info g
	join user u
	on g.user_id = u.id
	group by 1,2
)

select *
from table1
where grade_sum >= all(select grade_sum from table1)
# 或者最后一行写成grade_sum = (select max(grade_sum) from table1)



- **至此，牛客网91题SQL刷题结束，总体来说还是比较满意的！！完结撒花！！！！**